In [1]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

In [ ]:
batch_size = 32
model_name_or_path = "llange/xlm-roberta-large-spanish"
task = "mrpc"
peft_type = PeftType.LORA
device = "cuda"
num_epochs = 20
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
lr = 3e-4

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="right")
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

datasets = load_dataset("glue", task)
metric = evaluate.load("glue", task)


def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=None)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence1", "sentence2"],
)

# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

In [2]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("jorgeortizfuentes/bloom-1b1-spanish")
model = AutoModelForCausalLM.from_pretrained("jorgeortizfuentes/bloom-1b1-spanish", quantization_config=quant_config, device_map={"":0})

In [4]:
model.gradient_checkpointing_enable()

In [6]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(model,config)

In [20]:
input_ids = tokenizer("Hola buenas tardes", return_tensors="pt")
model.generate(**input_ids, max_new_tokens=2)

RuntimeError: 'weight' must be 2-D

In [13]:
input_ids

{'input_ids': [132417, 64614, 178990], 'attention_mask': [1, 1, 1]}